In [ ]:
import numpy as np
import plotly.graph_objects as go
# import soundfile as sf
import wave
import sys
from pathlib import Path
import sound_card_tools as sct
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.signal import spectrogram

#data=sct.load_csv_results_merged('output_phases.csv', fields=['frequency'])
data=sct.load_csv_results_merged('/home/user/data/sine_wave_120_sec_4KHz_phases.csv', fields=['frequency'])


In [ ]:
# plot the frequency using plotly
import plotly.graph_objects as go

# Create interactive frequency plot
fig = go.Figure()
fig.add_trace(go.Scatter(
    y=data['frequency'],
    mode='lines',
    name='Frequency'
))
fig.update_layout(
    title='Frequency vs Samples',
    xaxis_title='Sample Index',
    yaxis_title='Frequency [rad/s]'
)
fig.show()

In [ ]:
import numpy as np

def find_distinct_frequencies(frequencies, quantization_size=0.1):
    """
    Given an array of instantaneous frequency values that may drift around
    discrete setpoints, return a list of distinct (clustered) frequencies.

    We cluster frequencies so that any new value f is considered
    part of the current frequency cluster if:

        |f - current_cluster_center| <= quantization_size

    Otherwise, we start a new cluster. The center of each cluster is kept as
    the running average of all points in that cluster.

    Parameters
    ----------
    frequencies : array-like of float
        Array of instantaneous frequency measurements (e.g. from your signal processing).
    quantization_size : float, optional
        Tolerance used to decide whether a new frequency measurement belongs to
        the current cluster or starts a new one. Smaller values enforce stricter clustering,
        while larger values group more measurements together.

    Returns
    -------
    distinct_freqs : list of float
        The distinct (clustered) frequencies in ascending order. Each value
        is the average of all points that fell into that cluster.
    """
    # Convert to NumPy array (if not already) and sort
    freq_array = np.asarray(frequencies, dtype=np.float64)
    if freq_array.size == 0:
        return []

    freq_array.sort()

    # Each element of 'clusters' will be [running_mean, count]
    clusters = []

    for f in freq_array:
        if not clusters:
            # First cluster
            clusters.append([f, 1])
        else:
            current_mean, count = clusters[-1]
            if abs(f - current_mean) <= quantization_size:
                # Belongs to the current cluster -> update its center (mean)
                new_count = count + 1
                new_mean = (current_mean * count + f) / new_count
                clusters[-1] = [new_mean, new_count]
            else:
                # Start a new cluster
                clusters.append([f, 1])

    # Extract the final cluster centers
    distinct_freqs = [c[0] for c in clusters]
    return distinct_freqs


In [ ]:
data_unique = find_distinct_frequencies(data['frequency'], quantization_size=0.1)
data_unique = np.unique( np.round(data_unique).astype(int))

data_unique.sort()
print(data_unique)
data_unique_diff = np.diff(data_unique)
print(data_unique_diff)

data_unique_diff_unique = np.unique(data_unique_diff)
print(data_unique_diff_unique)




In [ ]:
def plot_frequency_histogram(freq_data, freq_min, freq_max, num_bins=100):
    """
    Plot histogram of frequencies within specified range
    
    Parameters
    ----------
    freq_data : array-like
        Frequency data in Hz
    freq_min : float
        Minimum frequency to include (Hz)
    freq_max : float
        Maximum frequency to include (Hz)
    num_bins : int, optional
        Number of histogram bins (default: 100)
    """
    # Filter data to specified range
    mask = (freq_data >= freq_min) & (freq_data <= freq_max)
    filtered_data = freq_data[mask]
    
    # print the number of data points
    print(f'Number of data points: {len(filtered_data)}')
    
    
    # Create histogram
    fig = go.Figure()
    fig.add_trace(go.Histogram(
        x=filtered_data,
        nbinsx=num_bins,
        name='Frequency Distribution'
    ))
    
    fig.update_layout(
        title=f'Frequency Histogram ({freq_min:.1f} Hz to {freq_max:.1f} Hz)',
        xaxis_title='Frequency [Hz]',
        yaxis_title='Count',
        showlegend=False
    )
    
    fig.show()

# Example usage:
freq_min = 3500.0  # Set minimum frequency
freq_max = 4500.0  # Set maximum frequency
# print number of data points
print(f'Number of data points: {len(data['frequency'])}')
plot_frequency_histogram(data['frequency'], freq_min, freq_max)


In [ ]:
# Example usage:
freq_diff_min = 0.0  # Set minimum frequency
freq_diff_max = 2500.0  # Set maximum frequency
# find the change in frequency between each data point
freq_diff = np.diff((data['frequency']))


plot_frequency_histogram(freq_diff, freq_diff_min, freq_diff_max, num_bins=2000)